In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from scipy.signal import butter, filtfilt
import glob
import pandas as pd
from tqdm import tqdm
import torch as th
from torch.utils.data import Dataset, DataLoader
import os

In [37]:
def process_df(signal_path: str, is_noise = False) -> pd.DataFrame:

    eq_signal_files = glob.glob(f'{signal_path}/**/*.npz', recursive=True)

    path_name = "earthquake"
    if is_noise:
        path_name = "noise"

    data = []
    for eq_path in tqdm(eq_signal_files):
        eq = np.load(eq_path, allow_pickle=True)
        Z = eq[path_name + '_waveform_Z']
        N = eq[path_name +'_waveform_N']
        E = eq[path_name +'_waveform_E']
        data.append({'Z': Z, 'N': N, 'E': E})

    df = pd.DataFrame(data)

    return df

In [ ]:
signal_path = "C:/Users/cleme/ETH/Master/DataLab/dsl-as24-challenge-3/data/signal/validation"
df_signal = process_df(signal_path, is_noise = False)
signal_path = "C:/Users/cleme/ETH/Master/DataLab/dsl-as24-challenge-3/data/noise/validation"
df_noise = process_df(signal_path, is_noise = True)

In [50]:
def plot_time_domain(df: pd.DataFrame, total: int):

    # Function to compute FFT and plot
    def plot_fft(signal, title, ax):

        n = len(signal)
        time = range(n)

        # Plot the magnitude of the FFT
        ax.plot(time, signal)
        ax.set_title(title)
        ax.set_xlabel('Time')
        ax.set_ylabel('Velocity')

    # Step 1: Create a figure with subplots
    fig, axs = plt.subplots(total, 3, figsize=(15, 5 * total))

    # Step 2: Compute and plot FFT for each signal in 'Z', 'N', 'E'
    for i in range(total):
        # Plot for Z signal
        plot_fft(df['Z'].iloc[i], f'Time Domain of Z Signal {i}', axs[i, 0])
        
        # Plot for N signal
        plot_fft(df['N'].iloc[i], f'Time Domain of N Signal {i}', axs[i, 1])
        
        # Plot for E signal
        plot_fft(df['E'].iloc[i], f'Time Domain of E Signal {i}', axs[i, 2])

    # Step 3: Display the plots
    plt.tight_layout()
    plt.show()

In [ ]:
plot_time_domain(df_signal, 3)

In [43]:
def plot_fourier(df: pd.DataFrame, N: int):

    # Function to compute FFT and plot
    def plot_fft(signal, title, ax):
        # Compute FFT
        fft_values = np.fft.fft(signal)
        fft_magnitude = np.abs(fft_values)  # Magnitude of the FFT

        # Frequency bins (corresponding to FFT output)
        freqs = np.fft.fftfreq(len(signal))

        # Plot the magnitude of the FFT
        ax.plot(freqs, fft_magnitude)
        ax.set_title(title)
        ax.set_xlabel('Frequency')
        ax.set_ylabel('Magnitude')

    # Step 1: Create a figure with subplots
    fig, axs = plt.subplots(N, 3, figsize=(15, 5 * N))

    # Step 2: Compute and plot FFT for each signal in 'Z', 'N', 'E'
    for i in range(N):
        # Plot for Z signal
        plot_fft(df['Z'].iloc[i], f'FFT of Z Signal {i}', axs[i, 0])
        
        # Plot for N signal
        plot_fft(df['N'].iloc[i], f'FFT of N Signal {i}', axs[i, 1])
        
        # Plot for E signal
        plot_fft(df['E'].iloc[i], f'FFT of E Signal {i}', axs[i, 2])

    # Step 3: Display the plots
    plt.tight_layout()
    plt.show()


In [ ]:
plot_fourier(df_signal)

In [ ]:
plot_fourier(df_noise)

In [2]:
class SeismicDataset(Dataset):

    def __init__(self, signal_folder_path: str, noise_folder_path: str, randomized = False):

        """
        Args:
            signal_folder_path (str): Path to earthquake signal folder containing .npz files.
            noise_folder_path (str): Path to noise folder containing .npz files.
        """

        self.signal_folder_path = signal_folder_path
        self.noise_folder_path = noise_folder_path
        self.eq_signal_files = glob.glob(f'{signal_folder_path}/**/*.npz', recursive=True)
        self.noise_files = glob.glob(f'{noise_folder_path}/**/*.npz', recursive=True)
        self.randomized = randomized

        self.signal_length = 6000


    def __len__(self) -> int:
        return len(self.eq_signal_files)

    def __getitem__(self, idx) -> tuple[th.Tensor, int, str]:

        eq_path = self.eq_signal_files[idx]
        eq = np.load(eq_path, allow_pickle=True)
        eq_name = (os.path.splitext(os.path.basename(eq_path)))[0]

        noise_to_small = True
        while noise_to_small:
            noise_idx = np.random.randint(0, len(self.noise_files))
            noise_path = self.noise_files[noise_idx]
            noise = np.load(noise_path, allow_pickle=True)
            if len(noise['noise_waveform_Z']) >= self.signal_length:
                noise_to_small = False
        
        noise_length = len(noise['noise_waveform_Z'])

        noise_name = (os.path.splitext(os.path.basename(noise_path)))[0]

        eq_start = 0
        noise_start = 0
        if self.randomized:
            eq_start = np.random.randint(low = 0, high = 6000)
            noise_start = np.random.randint(low = 0, high = max(noise_length - self.signal_length,1))

        Z_eq = eq['earthquake_waveform_Z'][eq_start:eq_start+self.signal_length]
        N_eq = eq['earthquake_waveform_N'][eq_start:eq_start+self.signal_length]
        E_eq = eq['earthquake_waveform_E'][eq_start:eq_start+self.signal_length]
        eq_stacked = np.stack([Z_eq, N_eq, E_eq], axis=0)

        Z_noise = noise['noise_waveform_Z'][noise_start:noise_start+self.signal_length]
        N_noise = noise['noise_waveform_N'][noise_start:noise_start+self.signal_length]
        E_noise = noise['noise_waveform_E'][noise_start:noise_start+self.signal_length]
        noise_stacked = np.stack([Z_noise, N_noise, E_noise], axis=0)

        j = np.random.choice([0, 1, 2])

        # tensor_normalized = eq_tensor / eqtensor.abs().max()

        p_wave_start = 6000 - eq_start
        # , p_wave_start, eq_name, noise_name
        
        noise = noise_stacked[j]
        eq = eq_stacked[j]

        noisy_eq = eq + noise

        return noisy_eq, eq

In [33]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs  # Nyquist frequency
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the filter
def apply_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y


def objective(lowcut, highcut, order):

    batch_size = 32
    fs = 100

    signal_folder = "C:/Users/cleme/ETH/Master/DataLab/dsl-as24-challenge-3/data/signal/train"
    noise_folder = "C:/Users/cleme/ETH/Master/DataLab/dsl-as24-challenge-3/data/noise/train"
    dataset = SeismicDataset(signal_folder, noise_folder)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    dataset_length = len(dataset)
    print(dataset_length)
    mean = 0

    #print(len(dataloader))

    MAX_BATCHES = 10

    for i, batch in enumerate(dataloader):
        
        if i >= MAX_BATCHES:
            break

        print(i)

        noisy_eq, ground_truth = batch
        noisy_eq_np = noisy_eq.numpy()

        filtered_signals = []
        for i in range(len(noisy_eq)):
            filtered_signal = apply_bandpass_filter(noisy_eq_np[i], lowcut, highcut, fs, order)
            filtered_signals.append(filtered_signal)

        # Convert the filtered signals back to a PyTorch tensor
        filtered_eq = th.tensor(np.array(filtered_signals))

        mean += (1/(MAX_BATCHES * 32)) * th.sum(th.square(ground_truth - filtered_eq))
    
    return mean

In [52]:
objective(2.0, 20.0, 4)

20230
0
1
2
3
4
5
6
7
8
9


tensor(0.5851, dtype=torch.float64)